## Rohan Gore 
###### rmg9725@nyu.edu
###### Collaborators: PerplexityAI
###

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col, explode, when, lit, udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import round
from pyspark.sql.window import Window
import pyspark.sql.functions as F


# Initialize Spark session
spark = SparkSession.builder.appName("E-Commerce Analysis").getOrCreate()

# Load datasets
customers_df = spark.read.json("customers.json")
orders_df = spark.read.json("orders.json")
order_items_df = spark.read.json("order_items.json")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/13 15:57:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Question 1

In [2]:

# calculating total amount per order and appending it ahead

order_totals = order_items_df.groupBy("order_id") \
    .agg(sum(col("quantity") * col("unit_price")).alias("order_total"))


# Join with orders and filter out cancelled orders
valid_orders = orders_df.filter(col("status") != "Cancelled") \
    .join(order_totals, "order_id")


In [3]:
# Calculate total spending per customer

customer_spending = valid_orders.groupBy("customer_id") \
    .agg(sum("order_total").alias("total_spent"))



# join that with customer data for providing more detailsin the final view 
customer_spending_with_details = customer_spending.join(customers_df, "customer_id") \
    .select("customer_id", "name", "tier", "total_spent") \
    .orderBy(col("total_spent").desc())


#rounding values
customer_spending_with_details = customer_spending_with_details.withColumn(
    "total_spent", 
    round(col("total_spent"), 2)
)


In [4]:
# Watch results
customer_spending_with_details.show()


+-----------+---------------+------+-----------+
|customer_id|           name|  tier|total_spent|
+-----------+---------------+------+-----------+
|          1|     John Smith|  Gold|    1488.92|
|          3|   Robert Brown|Bronze|     679.95|
|          2|   Mary Johnson|Silver|     577.91|
|          7| James Anderson|  Gold|     369.98|
|         10| Jennifer Clark|  Gold|     345.98|
|          8|Patricia Thomas|Silver|     209.97|
|          4|    Linda Davis|  Gold|     149.97|
|          5| Michael Wilson|Silver|      62.95|
+-----------+---------------+------+-----------+



- As the question does not tell how many "MOST SPENDING" customers to show in the final view, I am showing all the customers in the decreasing order.
- If the question is rephrased as "Show which X customers spend the most money overall" where X is a number, then we can view the result by executing the code:
  ```customer_spending_with_details.show(X)```

## Question 2

In [5]:
# Explode the categories array to get individual categories
categories_df = orders_df.filter(col("status") != "Cancelled") \
    .select("order_id", "customer_id", explode(col("categories")).alias("category"))


# Append/Join with customers to get tier information
customer_categories = categories_df.join(customers_df, "customer_id") \
    .select("tier", "category")


In [6]:


# Count category occurrences by tier

category_counts_by_tier = customer_categories.groupBy("tier", "category") \
    .count() \
    .orderBy("tier", col("count").desc())


In [7]:

# Map and find the most popular category for each tier

#doing this using a window function

window_spec = Window.partitionBy("tier").orderBy(col("count").desc())
top_categories_by_tier = category_counts_by_tier \
    .withColumn("rank", F.rank().over(window_spec)) \
    .filter(col("rank") == 1) \
    .select("tier", "category", "count")



In [8]:


# Categorizing just for proper visualization 

gold_categories = top_categories_by_tier.filter(col("tier") == "Gold")
print("Most Popular Categories for Gold Tier:")
gold_categories.show()

silver_categories = top_categories_by_tier.filter(col("tier") == "Silver")
print("Most Popular Categories for Silver Tier:")
silver_categories.show()

bronze_categories = top_categories_by_tier.filter(col("tier") == "Bronze")
print("Most Popular Categories for Bronze Tier:")
bronze_categories.show()



Most Popular Categories for Gold Tier:


+----+-----------+-----+
|tier|   category|count|
+----+-----------+-----+
|Gold|electronics|    3|
+----+-----------+-----+

Most Popular Categories for Silver Tier:
+------+-----------+-----+
|  tier|   category|count|
+------+-----------+-----+
|Silver|accessories|    2|
+------+-----------+-----+

Most Popular Categories for Bronze Tier:
+------+-----------+-----+
|  tier|   category|count|
+------+-----------+-----+
|Bronze|      books|    1|
|Bronze|electronics|    1|
|Bronze|       home|    1|
|Bronze| stationery|    1|
+------+-----------+-----+



- As the question does not tell how many "MOST POPULAR" categories for each tier are to be shown in the final view, I am showing ONLY the most popular category (top 1).
- If there are multiple categories with same number of occurences, then all of them will be shown, as in  the case of "Bronze Tier".
  
- If the question is rephrased as "Show the most popular X product categories for each customer tier (Gold, Silver, Bronze)" where X is a number, then we can view the result by altering the code:
  ```
  top_categories_by_tier = category_counts_by_tier \
    .withColumn("rank", F.rank().over(window_spec)) \
    .filter(col("rank") <= X) \ # <--------------------------------- change over here
    .select("tier", "category", "count")
  ```

## Question 3

In [9]:


# Define UDF for price classification
classify_price_udf = udf(lambda price: 
    "Budget" if price < 50.0 else 
    "Mid-range" if price < 200.0 else 
    "Premium", StringType())



In [10]:



# Apply price classification to order items
classified_items = order_items_df \
    .withColumn("price_category", classify_price_udf(col("unit_price"))) \
    .withColumn("item_total", col("quantity") * col("unit_price"))



# Join with orders (excluding cancelled) and customers
spending_by_price_category = classified_items \
    .join(orders_df.filter(col("status") != "Cancelled"), "order_id") \
    .join(customers_df, "customer_id") \
    .select("tier", "price_category", "item_total", "quantity") 



In [11]:

# Calculating total spending by --> Tier and Price Category

tier_price_spending = spending_by_price_category \
    .groupBy("tier", "price_category") \
    .agg(
        sum("item_total").alias("category_total"),
        sum("quantity").alias("product_count") 
    )


In [12]:

# Calculate total spending wrt NEW TIERS 

tier_total_spending = spending_by_price_category \
    .groupBy("tier") \
    .agg(sum("item_total").alias("tier_total"))


In [13]:

#  Join to calculate percentages

tier_price_distribution = tier_price_spending.join(tier_total_spending, "tier") \
    .withColumn("percentage", (col("category_total") / col("tier_total") * 100)) \
    .select("tier", "price_category", "category_total", "percentage", "product_count") \
    .orderBy("tier", "price_category")


# round both 

tier_price_distribution = tier_price_distribution.withColumn(
    "category_total", 
    round(col("category_total"), 2)
).withColumn(
    "percentage", 
    round(col("percentage"), 2)
)


In [14]:


# Categorizing just for proper visualization 


gold_distribution = tier_price_distribution.filter(col("tier") == "Gold")
print("Price Range Distribution for Gold Tier:")
gold_distribution.show()

silver_distribution = tier_price_distribution.filter(col("tier") == "Silver")
print("Price Range Distribution for Silver Tier:")
silver_distribution.show()

bronze_distribution = tier_price_distribution.filter(col("tier") == "Bronze")
print("Price Range Distribution for Bronze Tier:")
bronze_distribution.show()




Price Range Distribution for Gold Tier:


+----+--------------+--------------+----------+-------------+
|tier|price_category|category_total|percentage|product_count|
+----+--------------+--------------+----------+-------------+
|Gold|        Budget|        214.93|      9.13|            7|
|Gold|     Mid-range|        589.95|     25.05|            5|
|Gold|       Premium|       1549.97|     65.82|            3|
+----+--------------+--------------+----------+-------------+

Price Range Distribution for Silver Tier:
+------+--------------+--------------+----------+-------------+
|  tier|price_category|category_total|percentage|product_count|
+------+--------------+--------------+----------+-------------+
|Silver|        Budget|        220.88|     25.96|           12|
|Silver|     Mid-range|        629.95|     74.04|            5|
+------+--------------+--------------+----------+-------------+

Price Range Distribution for Bronze Tier:
+------+--------------+--------------+----------+-------------+
|  tier|price_category|category_

- This set of 3 tables for the **NEW TIERS** answers all the questions:
  1. Distribution of spending per customer tier across budget, mid-range, and premium products
        - Answered via:  values for NEW TIERS in columns: ```product_count``` ```category_total``` ```percentage```
  2. Comparison between customer tier expediture practices 
     - Answered via: values for NEW TIERS in columns: ```product_count``` ```category_total``` ```percentage```ALONG WITH separation of tables for easy lookup.
    

## Question 4

In [15]:
# Join with orders (excluding cancelled) and customers (similar to Q2.3 but with different columns)
# We can reuse the classified_items DataFrame from Q2.3


customer_spending_by_category = classified_items \
    .join(orders_df.filter(col("status") != "Cancelled"), "order_id") \
    .join(customers_df, "customer_id") \
    .select("customer_id", "name", "tier", "price_category", "item_total")


In [16]:

# Calculate total spending by customer, tier, and price category

customer_tier_price_spending = customer_spending_by_category \
    .groupBy("customer_id", "name", "tier", "price_category") \
    .agg(sum("item_total").alias("total_spent"))




In [17]:


# Rank customers within each tier-price category combination

window_spec = Window.partitionBy("tier", "price_category").orderBy(col("total_spent").desc())
ranked_customers = customer_tier_price_spending \
    .withColumn("rank", F.rank().over(window_spec))


In [18]:



# Filter to get top 2 customers in each NEW TIER price category combination

top_customers = ranked_customers \
    .filter(col("rank") <= 2) \
    .select("tier", "price_category", "customer_id", "name", "total_spent", "rank") \
    .orderBy("tier", "price_category", "rank")

top_customers = top_customers.withColumn(
    "total_spent", 
    round(col("total_spent"), 2)
)




In [19]:
# Categorizing just for proper visualization 


gold_customers = top_customers.filter(col("tier") == "Gold")
print("Top Gold Tier Customers:")
gold_customers.show()

silver_customers = top_customers.filter(col("tier") == "Silver")
print("Top Silver Tier Customers:")
silver_customers.show()

bronze_customers = top_customers.filter(col("tier") == "Bronze")
print("Top Bronze Tier Customers:")
bronze_customers.show()


Top Gold Tier Customers:
+----+--------------+-----------+--------------+-----------+----+
|tier|price_category|customer_id|          name|total_spent|rank|
+----+--------------+-----------+--------------+-----------+----+
|Gold|        Budget|          1|    John Smith|     138.95|   1|
|Gold|        Budget|         10|Jennifer Clark|      45.99|   2|
|Gold|     Mid-range|          1|    John Smith|     349.98|   1|
|Gold|     Mid-range|          7|James Anderson|     119.99|   2|
|Gold|       Premium|          1|    John Smith|     999.99|   1|
|Gold|       Premium|         10|Jennifer Clark|     299.99|   2|
+----+--------------+-----------+--------------+-----------+----+

Top Silver Tier Customers:
+------+--------------+-----------+---------------+-----------+----+
|  tier|price_category|customer_id|           name|total_spent|rank|
+------+--------------+-----------+---------------+-----------+----+
|Silver|        Budget|          2|   Mary Johnson|     107.95|   1|
|Silver|   

- These tables show the top 2 highest-spending customers within each **NEW TIER**.
- If there is only one person in the NEW TIER then it will show only that person 

In [20]:
print ("GG")

GG
